In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [893 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,088 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 2,239 kB in 3s (802 kB/s)
Reading pac

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
dataframe = spark.createDataFrame([
    (0, "Here"),
    (1, "Too"),
    (2, "Hi")], ["id", "words"]
)

dataframe.show()

+---+-----+
| id|words|
+---+-----+
|  0| Here|
|  1|  Too|
|  2|   Hi|
+---+-----+



In [ ]:
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)

sale_1 = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True)
sale_1.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
sale_1.createOrReplaceTempView('homeSales')
spark.sql("SELECT * FROM homeSales").show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
from pyspark.sql.functions import year, to_timestamp #code taken from https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.to_timestamp.html

sale_1 = sale_1.withColumn("date", to_timestamp(sale_1["date"], "yyyy-MM-dd"))
sale_1 = sale_1.withColumn("year", year(sale_1["date"]))
sale_1.show()

+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|                  id|               date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|
+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|f8a53099-ba1c-47d...|2022-04-08 00:00:00|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|2022|
|7530a2d8-1ae3-451...|2021-06-13 00:00:00|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|2021|
|43de979c-0bf0-4c9...|2019-04-12 00:00:00|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|2019|
|b672c137-b88c-48b...|2019-10-16 00:00:00|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|2019|
|e0726d4d-d595-407...|2022-01-08 00:00:00|      2017|424418|       3|        2|       2249|   138

In [ ]:
from pyspark.sql.functions import col, avg, round

question3 = sale_1.filter(col("bedrooms") == 4) \
    .groupBy("year") \
    .agg(round(avg("price"), 2).alias("Avg")) \
    .orderBy("year")

question3.show()

+----+---------+
|year|      Avg|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [ ]:
from pyspark.sql.functions import col, avg, round

question4 = sale_1.filter((col("bedrooms") == 3) & (col("bathrooms") == 3)) \
    .groupBy("DATE_BUILT") \
    .agg(round(avg("price"), 2).alias("Avg")) \
    .orderBy("DATE_BUILT")

question4.show()

+----------+---------+
|DATE_BUILT|      Avg|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [ ]:
from pyspark.sql.functions import col, avg, round

question5 = sale_1.filter((col("bedrooms") == 3) &
                                  (col("bathrooms") == 3) &
                                  (col("floors") == 2) &
                                  (col("sqft_living") >= 2000)) \
    .groupBy("date_built") \
    .agg(round(avg("price"), 2).alias("avg")) \
    .orderBy("date_built")

question5.show()

+----------+---------+
|date_built|      avg|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [ ]:
from pyspark.sql.functions import col, avg, round
import time

start_time = time.time()

question6 = sale_1.filter(col("price") >= 350000) \
    .groupBy("view") \
    .agg(round(avg("price"), 2).alias("avg")) \
    .orderBy("view", ascending=False)

question6.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|       avg|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|   9| 401393.34|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.48814868927001953 seconds ---


In [ ]:
spark.sql("CACHE TABLE homeSales") #Question7

DataFrame[]

In [ ]:
spark.catalog.isCached('homeSales') #Question8

True

In [ ]:
from pyspark.sql.functions import col, avg, round
from pyspark.sql.window import Window
from pyspark.sql import functions as F
import time

start_time = time.time()

question9 = sale_1.filter(col("price") >= 350000) \
    .groupBy("view") \
    .agg(round(avg("price"), 2).alias("avg")) \
    .withColumn("row_num", F.row_number().over(Window.orderBy(F.desc("view")))) \
    .orderBy("row_num") \
    .drop("row_num")

question9.show()

+----+----------+
|view|       avg|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|   9| 401393.34|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows



In [ ]:
sale_1.write.partitionBy("view").mode("overwrite").parquet("homeSales_partitioned") #Question10
p_df_p=spark.read.parquet("homeSales_partitioned") #Question11
p_df_p.createOrReplaceTempView("p_df_p") #Question12

In [ ]:
from pyspark.sql.functions import col, avg, round
import time

start_time = time.time()

question13 = sale_1.filter(col("price") >= 350000) \
    .groupBy("view") \
    .agg(round(avg("price"), 2).alias("avg")) \
    .orderBy(col("view").desc())

question13.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|       avg|
+----+----------+
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|   9| 401393.34|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.5403738021850586 seconds ---


In [ ]:
spark.catalog.uncacheTable("homeSales") #Question14
spark.catalog.isCached("homeSales") #Question15

False